In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.parse.corenlp import CoreNLPParser
from nltk.corpus import wordnet as wn
from nltk.probability import FreqDist
from collections import Counter
import math
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
# import xgboost as xgb

# SPACY IMPORT
import spacy
nlp = spacy.load("en_core_web_md")

In [14]:
# same readData from STS.py
def readData(fileName):

    first_sentence = []
    second_sentence = []
    score = []
    file = open(fileName, encoding="utf8")
    text = file.readline()
    text = file.read()
    # loop to extract a set of two sentences
    for sentence in text.split('\n'):
        # creating two separate lists of the sentences
        # '.rstrip('.') only removes the last period in the sentence
        first_sentence.insert(len(first_sentence),
                              (sentence.split('\t')[1].lower()).rstrip('.'))
        second_sentence.insert(len(first_sentence),
                               (sentence.split('\t')[2].lower()).rstrip('.'))
        # inserting the score as a separate lists
        score.insert(len(first_sentence), (sentence.split('\t')[3]))

    # print(first_sentence)
    return first_sentence, second_sentence, score


def preprocess(fileName):

    first_sentence, second_sentence, score = readData(fileName)
    first_sentence_tokens = []
    second_sentence_tokens = []

    # tokenizing and tagging
    first_sentence_tags = []
    second_sentence_tags = []

    for sentence in first_sentence:
        tokens = nltk.word_tokenize(sentence)
        first_sentence_tokens.insert(len(first_sentence_tokens), tokens)
        first_sentence_tags.insert(
            len(first_sentence_tags), nltk.pos_tag(tokens))
        # print(first_sentence_tokens)

    for sentence in second_sentence:
        tokens = nltk.word_tokenize(sentence)
        second_sentence_tokens.insert(len(second_sentence_tokens), tokens)
        second_sentence_tags.insert(
            len(second_sentence_tags), nltk.pos_tag(tokens))

        # print(second_sentence_tokens)

    # lemmatizing
    first_sentence_lemmas = []
    second_sentence_lemmas = []
    lemmatizer = WordNetLemmatizer()
    for sentence in first_sentence_tokens:
        sentence_components = []
        for token in sentence:
            lemmas = lemmatizer.lemmatize(token)
            sentence_components.insert(len(sentence_components), lemmas)
        first_sentence_lemmas.insert(
            len(first_sentence_lemmas), sentence_components)

    for sentence in second_sentence_tokens:
        sentence_components = []
        for token in sentence:
            lemmas = lemmatizer.lemmatize(token)
            sentence_components.insert(len(sentence_components), lemmas)
        second_sentence_lemmas.insert(
            len(second_sentence_lemmas), sentence_components)

    return first_sentence, second_sentence, score, first_sentence_tokens, second_sentence_tokens, first_sentence_lemmas, second_sentence_lemmas


In [17]:
s1_arr_train, s2_arr_train, scores_train, s1_tokens_train, s2_tokens_train, s1_lemmas_train,s2_lemmas_train = preprocess("./data/train-set.txt")

## Data Inspection

In [18]:
score_list = [0,0,0,0,0,0]
for s in scores_train:
    score_list[int(s)] += 1
for i in range(0, len(score_list)):
#     print(i, ': ', score_list[i], round(score_list[i]/len(score_list), 1), '%')
    print("% 1d: % 4d % 6.2f per" %(i, score_list[i], 100*score_list[i]/len(scores_train))) 

 0:    8   0.54 per
 1:   37   2.49 per
 2:   95   6.40 per
 3:  310  20.89 per
 4:  616  41.51 per
 5:  418  28.17 per


## Feature Engineering

This section includes all the code/functions to create features.

### Cosine Similarity (TF-IDF)

In [19]:
def calc_cosine_similarity(sentence1, sentence2):

    # remove the stopwords, transform into TF-IDF matrix, then
    tfidf_matrix = TfidfVectorizer(
        stop_words="english").fit_transform([sentence1, sentence2])
    cos_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
#     print(tfidf_matrix.toarray())

    cos_sim = cos_sim_matrix[0][1]

    return cos_sim

### Smooth Inverse Frequency (SIF)

In [20]:
def frequency_distribution(s1_tokens_array, s2_tokens_array):
    freq_dist = FreqDist()
    for i in range(len(s1_tokens_array)):
        for token in (s1_tokens_array[i] + s2_tokens_array[i]):
            freq_dist[token.lower()] += 1
    return freq_dist

In [21]:
freq_dist = frequency_distribution(s1_tokens_train, s2_tokens_train)
print(freq_dist.most_common(40))

[('the', 5169), (',', 3690), ('of', 2497), ('to', 2133), ('and', 1716), ('a', 1615), ('in', 1573), ('is', 891), ('that', 831), ('on', 820), ('for', 756), ('it', 587), ('this', 579), ('we', 531), ('with', 464), ('be', 459), ('by', 443), ('i', 425), ('which', 403), ('have', 384), ('not', 366), ('at', 343), ('as', 334), ('are', 333), ('has', 319), ('said', 316), ('was', 304), ('european', 287), ("'s", 280), ('from', 261), ('``', 252), ("''", 242), ('will', 233), ('.', 229), ('also', 223), ('its', 194), ('but', 193), ('would', 191), ('all', 188), ('percent', 187)]


In [22]:
def calc_sif_similarity(s1, s2, a = .001):
    vectorizer = CountVectorizer(stop_words="english")
    X = vectorizer.fit_transform([s1, s2])
    X_arr = X.toarray()
    sif_matrix = []
    for i in range(0, len(X_arr)):
        sif_arr = []
        for j in range(0, len(X_arr[i])):
            word = vectorizer.get_feature_names()[j]
            w = a / (a + freq_dist[word])
            v = X_arr[i][j]
            sif_arr.append(v*w)
        sif_matrix.append(sif_arr)
    sif_cos_sim_matrix = cosine_similarity(sif_matrix, sif_matrix)
    sif_cos_sim = sif_cos_sim_matrix[0][1]
    return sif_cos_sim

In [23]:
calc_sif_similarity('I like some apples', 'I like the pears')

1.4515545128534995e-10

### Simple Overlap

Unique words that are in both sentences divided by the total number of words in both sentences. Does not include stop words.

In [24]:
stop_words = set(stopwords.words('english'))
tokenized_sentence_list = s1_tokens_train+s2_tokens_train
words_filtered = []

# print(words)

# looking through I've noticed there are a number of stop-words that can be added to the set
stop_words.add(',')
stop_words.add('``')
stop_words.add("n't")

for tsl in tokenized_sentence_list:
    for w in tsl:
        if w not in stop_words and w not in words_filtered:
            words_filtered.append(w)

In [25]:
def remove_stopwords(token_list):
    blank_list = []
    for w in token_list:
        if w not in stop_words:
            blank_list.append(w)
    return blank_list

In [26]:
def remove_duplicate_tokens(token_list):
    blank_list = []
    for w in token_list:
        if w not in blank_list:
            blank_list.append(w)
    return blank_list

In [46]:
def calc_basic_overlap(s1_tokens, s2_tokens):
    s1_tokens = remove_stopwords(s1_tokens)
    s1_tokens = remove_duplicate_tokens(s1_tokens)

    s2_tokens = remove_stopwords(s2_tokens)
    s2_tokens = remove_duplicate_tokens(s2_tokens)
    
    overlap = 0
    encountered_indexes = []
    for word in (s1_tokens+s2_tokens):
        try:
            word_index = words_filtered.index(word)
            if word_index in encountered_indexes: # we know we have found an overlap
                overlap += 1
            encountered_indexes.append(word_index)
        except ValueError:
            # print(word + ' not found in lexicon. Skipping...')
            continue

    avg_sentence_len = len(s1_tokens+s2_tokens) / 2
    
    overlap_normlalized = overlap / avg_sentence_len
    return overlap, overlap_normlalized

### Synset Overlap

In [28]:
def calc_synset_overlap(s1_tokens, s2_tokens):
    s1_tokens = remove_stopwords(s1_tokens)
    s1_tokens = remove_duplicate_tokens(s1_tokens)

    s2_tokens = remove_stopwords(s2_tokens)
    s2_tokens = remove_duplicate_tokens(s2_tokens)
    
    print(s2_tokens)
    print(s1_tokens)

    s1_spread = []
    s2_spread = []
    
    for word in s1_tokens:
        for synset in wn.synsets(word):
            for i in range(0, len(synset.lemmas())):
                syn_word = synset.lemmas()[i].name()
                if syn_word not in s1_spread:
                    s1_spread.append(syn_word)

    for word in s2_tokens:
        for synset in wn.synsets(word):
            for i in range(0, len(synset.lemmas())):
                syn_word = synset.lemmas()[i].name()
                if syn_word not in s2_spread:
                    s2_spread.append(syn_word)         
    
    return calc_basic_overlap(s1_spread, s2_spread)
    
calc_synset_overlap(s1_tokens_train[0], s2_tokens_train[0])

['sources', 'close', 'sale', 'said', 'vivendi', 'keeping', 'door', 'open', 'bids', 'next', 'day', 'two']
['sources', 'close', 'sale', 'said', 'vivendi', 'keeping', 'door', 'open', 'bids', 'hoped', 'see', 'bidders', 'interested', 'individual', 'assets', 'team']
beginning not found in lexicon. Skipping...
root not found in lexicon. Skipping...
rootage not found in lexicon. Skipping...
source not found in lexicon. Skipping...
seed not found in lexicon. Skipping...
germ not found in lexicon. Skipping...
generator not found in lexicon. Skipping...
reservoir not found in lexicon. Skipping...
stopping_point not found in lexicon. Skipping...
finale not found in lexicon. Skipping...
finis not found in lexicon. Skipping...
finish not found in lexicon. Skipping...
conclusion not found in lexicon. Skipping...
closing_curtain not found in lexicon. Skipping...
close_up not found in lexicon. Skipping...
shut_down not found in lexicon. Skipping...
close_down not found in lexicon. Skipping...
come_toge

(46, 0.23173803526448364)

### Spacy Similarity

In [39]:
def calc_spacy_sim(s1, s2):
    s2 = nlp(s2)
    s1 = nlp(s1)
    return s1.similarity(s2)


## Pipeline

In this section we run the data through the pipeline to get it into the form necessary to create our models.

In [43]:
def pipeline(s1_array, s2_array, s1_tokens, s2_tokens, s1_lemmas, s2_lemmas):
    # TODO add a check to ensure the lengths of these arrays are the same
    # or add the basic processing to pipeline
    data = []
    for i in range(0, len(s1_array)):
        cos_sim = calc_cosine_similarity(s1_array[i], s2_array[i])
        sif_sim = calc_sif_similarity(s1_array[i], s2_array[i])
        w_overlap, w_norm_overlap = calc_basic_overlap(s1_tokens[i], s2_tokens[i])
        l_overlap, l_norm_overlap = calc_basic_overlap(s1_lemmas[i], s2_lemmas[i])
        spacy_sim = calc_spacy_sim(s1_array[i], s2_array[i])
#         syn_overlap, normalized_syn_overlap = calc_synset_overlap(s1_tokens[i], s2_tokens[i])
        data.append([i, w_norm_overlap, l_norm_overlap, sif_sim, cos_sim, spacy_sim])
    return data

In [54]:
data = pipeline(s1_arr_train, s2_arr_train, s1_tokens_train, s2_tokens_train, s1_lemmas_train, s2_lemmas_train)
print(data[0:5])

[[0, 0.6428571428571429, 0.6, 0.4043188683415115, 0.5949218057093537, 0.9728363156062543], [1, 0.631578947368421, 0.6, 0.37040524322972224, 0.474330706497194, 0.9396972779230695], [2, 0.5, 0.5, 0.1358693286767868, 0.392181175971253, 0.9275870020498693], [3, 0.7333333333333333, 0.6666666666666666, 0.6935512636502701, 0.668348418668298, 0.9731175952414417], [4, 0.24, 0.24, 4.979960298599938e-10, 0.12170566815950139, 0.8512029318853689]]


In [47]:
scores_train[0:5]

['4', '4', '3', '3', '2']

## Models

In this section we fit our feature set to a model.

### Decision Tree

In [48]:
dt_classifier = DecisionTreeClassifier(random_state=14, max_depth=8)
dt_classifier.fit(data,scores_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=14,
            splitter='best')

In [49]:
print(f'Nodes: {dt_classifier.tree_.node_count}')
print(f'Max Depth: {dt_classifier.tree_.max_depth}')
print(f'Accuracy: {dt_classifier.score(data, scores_train)}')

Nodes: 237
Max Depth: 8
Accuracy: 0.7028301886792453


### XGBoost

In [50]:
s1_arr_dev, s2_arr_dev, scores_dev, s1_tokens_dev, s2_tokens_dev = preprocess("./data/dev-set.txt")
dev_data = pipeline(s1_arr_dev, s2_arr_dev)
dev_predictions = dt_classifier.predict(dev_data)

xgboost_model = xgb.XGBRegressor(booster='gbtree', 
                       n_estimators=1000,
                       n_jobs=4,
                       learning_rate=.05,
                       max_depth=3,
                       random_state=42,
                       gamma=.05,
                       early_stopping_rounds = 5)



ValueError: too many values to unpack (expected 5)

In [ ]:
xgboost_model.fit(data, scores_train,
#                   eval_set=[(X_train, y_train), (X_test, y_test)],
                  eval_metric='logloss')

### Random Forest

In [51]:
rf_classifier = RandomForestClassifier(random_state=14, n_estimators=100)
rf_classifier.fit(data,scores_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=14, verbose=0, warm_start=False)

# Testing

In [56]:
s1_arr_dev, s2_arr_dev, scores_dev, s1_tokens_dev, s2_tokens_dev, s1_lemmas_dev,s2_lemmas_dev = preprocess("./data/dev-set.txt")
dev_data = pipeline(s1_arr_dev, s2_arr_dev, s1_tokens_dev, s2_tokens_dev, s1_lemmas_dev,s2_lemmas_dev)
dt_dev_predictions = dt_classifier.predict(dev_data)
rf_dev_predictions = rf_classifier.predict(dev_data)
dev_data[0:5]

[[0,
  0.42105263157894735,
  0.42105263157894735,
  0.0001332665010484381,
  0.4111217451482082,
  0.9552089581976497],
 [1,
  0.5714285714285714,
  0.8571428571428571,
  0.8164963796577869,
  0.7168117414430619,
  0.9945601722568282],
 [2,
  0.2857142857142857,
  0.2857142857142857,
  0.707106779665172,
  0.34464214103805474,
  0.9519508734395983],
 [3, 0.6, 0.6, 9.042867432144338e-05, 0.4112070550676187, 0.9244398882914221],
 [4, 0.25, 0.25, 1.0, 1.0, 1.0]]

In [57]:
# make sure our lengths match up
print(len(scores_dev))
print(len(dt_dev_predictions))
print(len(rf_dev_predictions))

1209
1209
1209


In [58]:
def calc_accuracy(predictions, scores):
    correct = 0
    for i in range(0, len(predictions)): 
        if predictions[i] == scores[i]:
            correct += 1
    return correct / len(predictions) 



    
print('DECISION TREE: ', calc_accuracy(dt_dev_predictions, scores_dev))
print('RANDOM FOREST: ', calc_accuracy(rf_dev_predictions, scores_dev))

DECISION TREE:  0.22001654259718775
RANDOM FOREST:  0.2878411910669975
